# Translate text with Azure AI Translator service

The Translator service enables you to create intelligent apps and services that can translate text between languages.

- Provision a Translator resource
- Understand language detection, translation, and transliteration
- Specify translation options
- Define custom translations

# Importing libraries

In [1]:
from dotenv import load_dotenv
import os

In [2]:
from azure.ai.translation.text import *
from azure.ai.translation.text.models import InputTextItem

In [3]:
from functools import lru_cache

# Loading configurations

In [4]:
load_dotenv()
translatorRegion = os.getenv('TRANSLATOR_REGION')
translatorKey = os.getenv('TRANSLATOR_KEY')

True

# Initializing client

In [5]:
credential = TranslatorCredential(translatorKey, translatorRegion)
client = TextTranslationClient(credential)

# Check language support

In [6]:
@lru_cache(maxsize=None)
def get_supported_languages():
  return client.get_languages(scope="translation")


def check_language_support(target_language):
  languages_response = get_supported_languages()
  supported_languages = languages_response.translation.keys()

  print(f"{len(supported_languages)} languages supported.")
  if target_language in supported_languages:
    print(f"{target_language} ({
          languages_response.translation[target_language]['name']}) is a supported language.")
  else:
    print(f"{target_language} is not a supported language.")

  print()

In [7]:
check_language_support('es')

135 languages supported.
es (Spanish) is a supported language.



# Translation API call

In [8]:
@lru_cache(maxsize=None)
def get_translation(input_text, target_language):
  input_text_elements = [{"text": input_text}]
  translation_response = client.translate(
      content=input_text_elements, to=[target_language])
  translation = translation_response[0] if translation_response else None
  return translation

In [9]:
def translate_text(input_text, target_language):
  translation = translation = get_translation(input_text, target_language)

  if translation:
    source_language = translation.detected_language
    for translated_text in translation.translations:
      print(f"Input {source_language}: '{input_text}'")
      print(f"{source_language} -> {translated_text.to}")
      print(f"Translated: '{translated_text.text}'")
      print()

# Main logic

In [10]:
target_language = 'es'
input_text = 'Sanika is a star ⭐'

check_language_support(target_language)
translate_text(input_text, target_language)

135 languages supported.
es (Spanish) is a supported language.

Input {'language': 'en', 'score': 1.0}: 'Sanika is a star ⭐'
{'language': 'en', 'score': 1.0} -> es
Translated: 'Sanika es una estrella ⭐'

